The conclusion from this analysis is that pytorch is way better at solving Ah=b then numpy and scipy maximinzation methods

In [1]:
import numpy as np
import pandas as pd
def findA(x0, x1, endpoints):
    '''
    Endpoints will have length m+1, where m is the number of bins for which we will find the elongation rates for.
    Endpoitns should have the first element equal to the start of the gene. The last element tends to be np.inf
    bc we may assume that the reads can run to beyond the end of the genes.
    :param x0:
    :param x1:
    :param endpoints:
    :return:
    '''
    assert len(x0)==len(x1), "x0 and x1 must have the same length"
    n = len(x0)
    m = len(endpoints) - 1  # endpoints include the first position in the gene, so the number of features is len(endpoints)-1, with the last feature corresponding to the run through region
    A = np.zeros((n, m))  # this is the coefficient matrix that we will construct. Each entry corresponds to the length of the portion within the segment between x0 and x1 that falls within the feature of endpoints
    for sample_idx in range(n):
        this_x0 = x0[sample_idx]
        this_x1 = x1[sample_idx]
        for i in range(m):
            if this_x0 < endpoints[i]:
                break
            if this_x0 > endpoints[i + 1]:  # this entry starts after the end of this feature
                continue
            if this_x1 < endpoints[i]:  # this entry ends before the start of this feature
                break  # no need to continue since A is initally filled with zeros
            if this_x0 >= endpoints[i] and this_x0 < endpoints[i + 1]:  # this entry starts within this feature
                if this_x1 > endpoints[i + 1]:  # this entry ends after the end of this feature
                    A[sample_idx, i] = endpoints[i + 1] - this_x0
                    this_x0 = endpoints[i + 1]
                    continue  # go to the next feature
                else:  # this entry ends within this feature
                    A[sample_idx, i] = this_x1 - this_x0
                    break  # no need to continue to the following features since A is initally filled with zeros
    return A


In [2]:
fn = '/gladstone/engelhardt/home/hvu/source/RNA_rates/splicingrates/simulations/tests/time_0.csv.gz'
df = pd.read_csv(fn, header=0, index_col=None, sep = '\t')
x0 = df.x0
x1 = df.x1
fixed_breaks = np.arange(0, 4.81, 0.2)
fixed_breaks = np.append(fixed_breaks, np.inf)
A= findA(x0, x1, fixed_breaks)
b = np.ones(A.shape[0]) * 5

In [9]:
# Solve using numpy's least squares
h_inv, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

print(1/h_inv)

[0.32288387 0.36150963 2.11352436 0.44213009 0.17069606 0.64227488
 0.35421053 0.16343612 0.70476245 0.24352368 0.63561532 0.42460633
 0.23427922 0.51260234 0.41611364 0.21128842 0.35567557 0.21304939
 0.87780313 0.42324577 0.2067653  0.57357912 0.60617981 0.20837057
 0.32998998]


In [9]:
from scipy.linalg import lstsq

# Solve using scipy's least squares
h_inv, residuals, rank, s = lstsq(A, b)

print(1/h_inv)


[0.32288387 0.36150963 2.11352436 0.44213009 0.17069606 0.64227488
 0.35421053 0.16343612 0.70476245 0.24352368 0.63561532 0.42460633
 0.23427922 0.51260234 0.41611364 0.21128842 0.35567557 0.21304939
 0.87780313 0.42324577 0.2067653  0.57357912 0.60617981 0.20837057
 0.32998998]


In [13]:
import torch

# Initialize h with requires_grad=True
h_inv = torch.randn(len(fixed_breaks)-1, 1, requires_grad=True)
At = torch.tensor(A).float()
bt = torch.tensor(b).float().unsqueeze(1)
# Define the optimizer
optimizer = torch.optim.Adam([h_inv], lr=0.01)

# Define the loss function
loss_fn = torch.nn.MSELoss()

# Training loop
num_epochs = 5000
for epoch in range(num_epochs):
    optimizer.zero_grad()  # Zero the gradients
    Ah = torch.mm(At, h_inv)  # Compute Ah
    loss = loss_fn(Ah, bt)  # Compute the loss
    loss.backward()  # Backpropagate the loss
    optimizer.step()  # Update h

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

h = 1 / h_inv.detach().numpy().flatten()
print(h)
print(h.shape)


Epoch 0, Loss: 24.965911865234375
Epoch 100, Loss: 12.043517112731934
Epoch 200, Loss: 5.085347652435303
Epoch 300, Loss: 1.8775923252105713
Epoch 400, Loss: 0.623275637626648
Epoch 500, Loss: 0.20617185533046722
Epoch 600, Loss: 0.08520952612161636
Epoch 700, Loss: 0.05272015929222107
Epoch 800, Loss: 0.04339741915464401
Epoch 900, Loss: 0.039718419313430786
Epoch 1000, Loss: 0.03749248757958412
Epoch 1100, Loss: 0.03576952964067459
Epoch 1200, Loss: 0.0343201644718647
Epoch 1300, Loss: 0.033072683960199356
Epoch 1400, Loss: 0.03199253976345062
Epoch 1500, Loss: 0.031055303290486336
Epoch 1600, Loss: 0.030239850282669067
Epoch 1700, Loss: 0.029527030885219574
Epoch 1800, Loss: 0.028899643570184708
Epoch 1900, Loss: 0.02834259532392025
Epoch 2000, Loss: 0.027843181043863297
Epoch 2100, Loss: 0.02739091031253338
Epoch 2200, Loss: 0.026977432891726494
Epoch 2300, Loss: 0.02659621648490429
Epoch 2400, Loss: 0.026242202147841454
Epoch 2500, Loss: 0.025911519303917885
Epoch 2600, Loss: 0.02